## Sentiment Analysis (clean_tweets) - VADER

VADER (Valence Aware Dictionary and Sentiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media.

VADER takes into account:
negations and contractions (not good, wasn’t good) Punctuation (good!!!), CAPS, emotes :), emojis Intensificators (very, kind of), acronyms ‘lol’ Scores between -1.0 (negative) and 1.0 (positive)

We will use this sentiment analysis of the tweets to calculate a score that will represent the importance of each tweet.

In [2]:
from time import sleep
import json
import io
import re
import numpy as np
from tqdm import tqdm
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm import tnrange, tqdm_notebook, tqdm

from sklearn import preprocessing
import matplotlib.pyplot as plt

In [3]:
directory = '~/PycharmProjects/tfm_hugopobil'
tweets = pd.read_csv(f'{directory}/data/tweets_clean.csv', low_memory=False)

In [5]:
print(tweets.shape)
tweets.head()

(23475, 14)


,index,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,1545110,Muhammad Sa'ad,NaN,Dota 2,2021-08-12 09:52:26,8.0,1089.0,1403.0,False,2021-10-28 12:30:44,"Airdrop is live , dont be late sir :)\n\n\n\n\...","['Airdrop', 'Airdrops', 'Airdropinspector', 'B...",Twitter for Android,False
1,1431064,Kikan4444,NaN,share the positive vibes of the universe 🧲. 💜 ...,2021-07-09 15:46:05,346.0,214.0,1534.0,False,2021-10-22 05:47:45,something Big is coming \n$KLV Klever 💜🌟\n\nBT...,"['Klever', 'BTC', 'ETH', 'BNB', 'TRX']",Twitter for Android,False
2,1070632,Kripto Tiger 🇹🇷,NaN,#bsc #bscgem #gem #shrew #gmrfinance #nftart,2020-11-07 07:37:33,164.0,298.0,5577.0,False,2021-08-16 09:58:44,Easy money with \n\n bsc bnb BinanceSmartChain...,"['bsc', 'bnb', 'BinanceSmartChain', 'binance',...",Twitter Web App,False
3,1276832,Sheesh | @NanoDogecoin | INDC,NaN,#Staking #INDC!\n#Stake #NanoDogecoin & #earn:...,2011-02-10 04:39:28,151.0,453.0,6228.0,False,2021-10-19 02:09:26,NanoDogecoin is the King👑of the BSC Blockchain...,"['NanoDogecoin', 'King', 'BSC', 'Blockchain', ...",Twitter for Android,False
4,1980309,CoinMarketDaddy (CMD),United Kingdom,CMD stands for Coin Market Daddy & Was Built B...,2018-04-19 02:19:06,2833.0,1.0,118.0,False,2021-12-30 16:03:28,"Bitcoin News Roundup for June 17, 2020 cryptoc...","['cryptocurrencies', 'bitcoin', 'crypto', 'cry...",SocialBee.io v2,False


In [7]:
analyzer = SentimentIntensityAnalyzer()
# Compound is the score given to the intensity of sentiment detected in tweets

compound = []

for i, s in enumerate(tqdm(tweets['text'],
                           position = 0,
                           leave = True)):

    # Variable vs can be modified to test different sentiment analysis
    
    vs = analyzer.polarity_scores(str(s))


    compound.append(vs['compound'])

tweets['compound'] = compound
tweets.head()

100%|██████████| 23475/23475 [00:03<00:00, 7387.99it/s]


,index,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,compound
0,1545110,Muhammad Sa'ad,NaN,Dota 2,2021-08-12 09:52:26,8.0,1089.0,1403.0,False,2021-10-28 12:30:44,"Airdrop is live , dont be late sir :)\n\n\n\n\...","['Airdrop', 'Airdrops', 'Airdropinspector', 'B...",Twitter for Android,False,0.4588
1,1431064,Kikan4444,NaN,share the positive vibes of the universe 🧲. 💜 ...,2021-07-09 15:46:05,346.0,214.0,1534.0,False,2021-10-22 05:47:45,something Big is coming \n$KLV Klever 💜🌟\n\nBT...,"['Klever', 'BTC', 'ETH', 'BNB', 'TRX']",Twitter for Android,False,0.6369
2,1070632,Kripto Tiger 🇹🇷,NaN,#bsc #bscgem #gem #shrew #gmrfinance #nftart,2020-11-07 07:37:33,164.0,298.0,5577.0,False,2021-08-16 09:58:44,Easy money with \n\n bsc bnb BinanceSmartChain...,"['bsc', 'bnb', 'BinanceSmartChain', 'binance',...",Twitter Web App,False,0.4404
3,1276832,Sheesh | @NanoDogecoin | INDC,NaN,#Staking #INDC!\n#Stake #NanoDogecoin & #earn:...,2011-02-10 04:39:28,151.0,453.0,6228.0,False,2021-10-19 02:09:26,NanoDogecoin is the King👑of the BSC Blockchain...,"['NanoDogecoin', 'King', 'BSC', 'Blockchain', ...",Twitter for Android,False,0.8475
4,1980309,CoinMarketDaddy (CMD),United Kingdom,CMD stands for Coin Market Daddy & Was Built B...,2018-04-19 02:19:06,2833.0,1.0,118.0,False,2021-12-30 16:03:28,"Bitcoin News Roundup for June 17, 2020 cryptoc...","['cryptocurrencies', 'bitcoin', 'crypto', 'cry...",SocialBee.io v2,False,0.0000


### Calculate score for each tweet

In [16]:
scores = []
for i, s in tqdm(tweets.iterrows(),
                 total=tweets.shape[0],
                 position=0,
                 leave=True):

    try:
    # Scores are calculated using the relevance of the tweeter poster
        scores.append(s["compound"] * (int(s["user_followers"])) * ((int(s["user_favourites"])+1) / int(s['user_followers']+1)) * \
                      (int(s["is_retweet"])+1))
    except:
        scores.append(np.nan)

tweets["score"] = scores

100%|██████████| 23475/23475 [00:00<00:00, 34464.48it/s]


### Save to local:

In [17]:
tweets.to_csv('tweets_scores.csv')